In [52]:
import pandas as pd
import numpy as np
import collections
import json
import re

In [76]:
def ascii_converter(text):
    patterns = [
        (r"â\x80\x93", " to "),
        (r"\xa0", " ")
    ]
    for (pattern, replacement) in patterns:
        text = re.sub(pattern, replacement, text)

    return text

In [13]:
root = "../../../data/interim/"

df_bert = pd.read_csv(root + "DF_GoogleForms_BERT_AllData.csv", 
                        index_col=0)
df_jacc = pd.read_csv(root + "DF_GoogleForms_Jaccard_AllData.csv",
                        index_col=0)

# df_jacc["SIM"] = "Jacc"
# df_bert["SIM"] = "Bert"

In [78]:
df_bert_sample = df_bert[df_bert["Sim Sum"] > 0.90]\
    .sample(n=20, replace=False, random_state=333)

df_jacc_sample = df_jacc[df_jacc["Jaccard Sum"] > 0.050]\
    .sample(n=20, replace=False, random_state=333)

In [79]:
df_jacc_sample

,Species,Main Trait,GT Main Trait,GT Sub Traits,1,2,3,4,5,GoogleSentence,Jaccard Sum
7746,Ziziphus abyssinica,flower color,flower color,"['White flower', 'Yellow coloured flower']",Flower yellow-green.,Flower yellow-green Provided by:.,NaN,NaN,NaN,Flower color: white flower,0.069444
1319,Cadaba glandulosa,plant type,plant type,['Bush'],Definition: whole plant arborescent.,NaN,NaN,NaN,NaN,Plant type: bush,0.055556
2852,Erythrophleum africanum,fruit,fruit,['Pod'],"Fruit a flat woody pod up to 17 cm long, dehis...","Fruit a flat, straight, dehiscent pod, ellipti...",NaN,NaN,NaN,Fruit: pod,0.055455
6553,Securidaca longipedunculata,bark,bark,"['Smooth bark', 'Finely fissured bark']","It has pale grey, smooth bark with leaves that...",The violet tree is a small to medium-sized tre...,NaN,NaN,NaN,Bark: smooth bark,0.053571
1248,Bridelia micrantha,sexuality,sexuality,['Axillary sexuality'],Inflorescence a small axillary fascicle.,NaN,NaN,NaN,NaN,Sexuality: axillary sexuality,0.083333
6031,Psidium guajava,bark,bark,"['Smooth bark', 'Finely fissured bark', 'Bark ...",The bark is smooth and reddish brown in color.,"Bark gray, smooth, peeling in strips.","Bark smooth, green to red-brown, peeling off i...","The older stems are covered in a smooth, light...",This sometimes gives the trunks a mottled appe...,Bark: smooth bark,0.057404
6215,Punica granatum,petiole,petiole,['Petiolated leaf'],Deciduous Leaf Fall Color:.,NaN,NaN,NaN,NaN,Petiole: petiolated leaf,0.055556
2943,Eucalyptus tereticornis,fruit,fruit,['Capsule'],"The fruit is a woody, hemispherical capsule 2â...","The fruit is a woody, hemispherical capsule 2â...","Fruit a capsule, 6-8 x 4-5 mm, pedicellate, tr...","Fruit pedicellate , hemispherical, 0.3â0.5 c...",Species in series Liberivalvae all have fruit ...,Fruit: capsule,0.054171
4775,Lawsonia inermis,phyllotaxis,phyllotaxis,['Opposite phyllotaxis'],The leaves grow opposite each other on the stem.,NaN,NaN,NaN,NaN,Phyllotaxis: opposite phyllotaxis,0.055556
1291,Burkea africana,fruit,fruit,['Pod'],The fruit is a grey woody pod.,"Fruit an elliptical, strongly flattened pod 3â...",NaN,NaN,NaN,Fruit: pod,0.056250
